In [1]:
using Plots
using SparseArrays
using LinearAlgebra
using BenchmarkTools
using DifferentialEquations

In [2]:
function set_initial_condition_1(x)
    A = 1.
    l = 80
    #-A * sin((pi / l ) * x)
    0 * x + 10
end

set_initial_condition_1 (generic function with 1 method)

# A) Define function 

$$ 

\frac{\partial ^2 W}{\partial t^2} = \alpha \frac{\partial ^4 W}{\partial x^4} 
$$
$$
\Leftrightarrow

\frac{\partial ^2 W}{\partial t^2} = \alpha  \times f(ddu,du,u,p,t)
$$

With the function $ f(du,u,p,t) $ for i in [1,N+1] : 

$$
ddu[1] = \frac{u_2 - u_1}{dx}
$$

$$
ddu[2] = \frac{u_3 - 2u_2 + u_1}{dx^2}
$$

$$
ddu[3:length(u)-2] = \frac{u_{i+2} - 4u_{i+1} + 6u_{i} -4u_{i-1} + u_{i-2}}{dx^4}
$$

$$
ddu[N] = \frac{u_{N+1}- 2u_N + u_{N-1}}{dx^2}
$$

$$
ddu[N+1] = \frac{u_{N+1} - u_N}{dx}
$$

Morover, the CFL condition is $ \frac{dx^2}{2C} > dt $


In [1]:
function matrix(n)
    #Parameter
        l = 80.#length 
        dx = l/n
        E = 1.9 * 10^7
        I = 117.8 
       # α = E * I / dx^4
       α = E *I / dx^4
    
        
    
    # Build the matrix
        A = spdiagm( -1 => -4 *α* ones(n-1), -2 =>  α* ones(n-2), 0 =>  6  *α* ones(n)   , 1 => - 4 *α* ones(n-1), 2 =>   α*ones(n-2) )
    
    # Coefficient that change in the  matrix 
        A[1,2] = -1/dx
        A[1,1]=  1/dx
        A[1,3] = 0.

        A[2,1] = 1/dx^2
        A[2,2] = -2/dx^2
        A[2,3] = 1/dx^2
        A[2,4] = 0.

        A[n-1, n-3] = 0.
        A[n-1,n-2] = 1/dx
        A[n-1,n-1] =-2/dx
        A[n-1,n] = 1/dx
        
        A[n,n-2] = 0.
        A[n,n-1] = -1/dx
        A[n,n]= 1/dx
    
    #time 135.690 microseconds and 65 allocations for n= 2000
        return A
    end

matrix (generic function with 1 method)

In [5]:
function biharmonic!(ddu,du,u,p,t)
    n = 81
    A = matrix(n)
    ddu .=  A * u
end

biharmonic! (generic function with 1 method)

In [6]:
N = 81
L = 80.0
dx = 1
x = Vector(0.0:dx:L)


init_t = set_initial_condition_1.(x)
init_dt = zeros(length(init_t))   # no initial speed


t_begin  = 0.0
t_end = 10
tspan = (t_begin, t_end)


prob = SecondOrderODEProblem(biharmonic!,init_dt, init_t, tspan)
sol = DifferentialEquations.solve(prob)

In [ ]:

plot(sol(10), label = "1000s")
plot!(sol(8), label = "800s")
plot!(sol(7), label = "700s")
plot!(sol(6), label = "600s")
plot!(sol(4), label = "400s")
plot!(sol(3), label="300s")
plot!(sol(2), label = "200s")
plot!(sol(t_begin), label = "0s")